In [2]:
#Install pysqlite3 for python and import pandas to use later
!pip install pysqlite3
from pysqlite3 import dbapi2 as sqlite3
print(sqlite3.sqlite_version)
import pandas as pd
from IPython.display import display, HTML

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 714.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysqlite3: filename=pysqlite3-0.5.2-cp310-cp310-linux_x86_64.whl size=151112 sha256=a1ddaa98c24855316e9f124f8560532cacf9d6be7b4f5aa25fe8c8113a3d9418
  Stored in directory: /root/.cache/pip/wheels/91/91/77/1912a316aad35d0edcb42ee1aa9b7bc07d7190a5798d378805
Successfully built pysqlite3
3.37.2


In [3]:
dbname = "sanfransico.db"

def printSqlResults(cursor, tblName):
  try:
    df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
    display(HTML("<b><font color=Green> " + tblName + "</font></b>" + df.to_html(index=False)))
  except:
    pass

def runSql(caption, query):
  conn = sqlite3.connect(dbname) # Connect to the database
  cursor = conn.cursor() # Create a cursor (think: it's like a "pointer")
  cursor.execute(query) # Execute the query
  printSqlResults(cursor, caption) # Print the results
  conn.close()

def runStepByStepSql(query, fromline):
  lines = query.strip().split('\n')
  for lineidx in range(fromline, len(lines)):
    partial_query = '\n'.join(lines[:lineidx])
    caption = 'Query till line:' +  partial_query
    runSql(caption, partial_query + ';')

In [4]:
# Connect to database (creates the file if it doesn't exist)
"""
1. Connections: A connection represents a connection to a database through
which we can execute SQL queries. The dbname here specifies the database.
In SQLlite, if the DB doesn't exist, it will be created.
2. Cursors: A cursor is an object associated with a database connection.
It allows you to execute SQL queries, fetch query results.
"""
conn = sqlite3.connect(dbname)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS Service_Requests_311 (
  unique_key INTEGER NOT NULL,
  created_date TIMESTAMP NOT NULL,
  closed_date TIMESTAMP NOT NULL,
  resolution_action_date TIMESTAMP,
  status VARCHAR(100) NOT NULL,
  status_notes VARCHAR(100) NOT NULL,
  agency_name VARCHAR(100) NOT NULL,
  category VARCHAR(100) NOT NULL,
  complaint_type VARCHAR(100) NOT NULL,
  incident_address VARCHAR(100) NOT NULL,
  neighborhood VARCHAR(100) NOT NULL,
  latitude FLOAT,
  longitude FLOAT,
  policy_district VARCHAR(100) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Bikeshare_Stations (
  station_id INTEGER NOT NULL,
  name STRING,
  latitude FLOAT64,
  longitude FLOAT64,
  dockcount INTEGER,
  landmark STRING,
  installation_date DATE
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Bikeshare_Status (
  station_id INTEGER NOT NULL,
  bikes_available INTEGER NOT NULL,
  docks_available INTEGER NOT NULL,
  time TIMESTAMP NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Bikeshare_Trips (
  trip_id INTEGER NOT NULL,
  duration_sec INTEGER NOT NULL,
  start_date TIMESTAMP,
  start_station_name VARCHAR(100) NOT NULL,
  start_station_id INTEGER,
  end_date TIMESTAMP,
  end_station_name VARCHAR(100) NOT NULL,
  end_station_id INTEGER,
  bike_number INTEGER NOT NULL,
  zip_code VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Film_Locations (
  title VARCHAR(100) NOT NULL,
  release_year INTEGER,
  locations VARCHAR(100),
  production_company VARCHAR(100) NOT NULL,
  director VARCHAR(100) NOT NULL,
  writer VARCHAR(100) NOT NULL,
  actor_1 VARCHAR(100) NOT NULL,
  actor_2 VARCHAR(100) NOT NULL
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS SFFD_Service_Calls (
  call_number INTEGER NOT NULL,
  unit_id VARCHAR(100),
  incident_number INTEGER NOT NULL,
  call_type VARCHAR(100),
  call_date DATE,
  response_timestamp TIMESTAMP,
  on_scene_timestamp IMTESTAMP,
  address VRCHAR(100),
  city VARCHAR(100),
  zipcode_of_incident VARCHAR(100),
  station_area VARCHAR(100)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS SFPD_Incidents (
  unique_key INTEGER NOT NULL,
  category VARCHAR(100) NOT NULL,
  descript VARCHAR(100) NOT NULL,
  dateofweek VARCHAR(100) NOT NULL,
  pddistrict VARCHAR(100) NOT NULL,
  resolution VARCHAR(100) NOT NULL,
  address VARCHAR(100) NOT NULL,
  latitude FLOAT,
  longitude FLOAT,
  timestamp TIMESTAMP
);
""")


# Commit changes and close the connection
conn.commit()
conn.close()

In [5]:
# Connect to database again and insert sample data
conn = sqlite3.connect(dbname)
sqlite3.enable_callback_tracebacks(True)

cursor = conn.cursor()
cursor.execute("delete from Service_Requests_311;")
cursor.execute("delete from Bikeshare_Stations;")
cursor.execute("delete from Bikeshare_Status;")
cursor.execute("delete from Film_Locations;")
cursor.execute("delete from SFPD_Incidents;")
cursor.execute("delete from SFFD_Service_Calls;")

cursor.execute("""
INSERT INTO Bikeshare_Stations(station_id, name, latitude, longitude, dockcount, landmark, installation_date)
VALUES
  (1, 'Japantown', 37.348742, -121.894715, 15, 'San Jose', '2020-01-01'),
  (2, 'Station 2', 37.7766, -122.3956, 15, 'San Francisco', '2020-01-05'),
  (3, 'Station 3', 37.7749, -122.4194, 20, 'San Francisco', '2020-03-15'),
  (4, 'Station 4', 37.7755, -122.4045, 18, 'San Francisco', '2020-04-02'),
  (5, 'Station 5', 37.7832, -122.3986, 22, 'San Francisco', '2020-05-10'),
  (6, 'Station 6', 37.7910, -122.4016, 19, 'San Francisco', '2020-06-20'),
  (7, 'Station 7', 37.7948, -122.4041, 21, 'San Francisco', '2020-07-01'),
  (8, 'Station 8', 37.7892, -122.4099, 17, 'San Francisco', '2020-08-14'),
  (9, 'Station 9', 37.7856, -122.4143, 16, 'San Francisco', '2020-09-25'),
  (10, 'Station 10', 37.7814, -122.4219, 20, 'San Francisco', '2020-10-05'),
  (11, 'Station 11', 37.7779, -122.4263, 18, 'San Francisco', '2020-11-18'),
  (12, 'Station 12', 37.7733, -122.4317, 23, 'San Francisco', '2020-12-30'),
  (13, 'Station 13', 37.7687, -122.4351, 14, 'San Francisco', '2021-01-11'),
  (14, 'Station 14', 37.7621, -122.4384, 16, 'San Francisco', '2021-02-22'),
  (15, 'Station 15', 37.7565, -122.4418, 19, 'San Francisco', '2021-03-05'),
  (16, 'Station 16', 37.7509, -122.4452, 21, 'San Francisco', '2021-04-17'),
  (17, 'Station 17', 37.7453, -122.4486, 20, 'San Francisco', '2021-05-29'),
  (18, 'Station 18', 37.7397, -122.4520, 18, 'San Francisco', '2021-06-10'),
  (19, 'Station 19', 37.7341, -122.4554, 17, 'San Francisco', '2021-07-22'),
  (20, 'Station 20', 37.7285, -122.4588, 15, 'San Francisco', '2021-08-04'),
  (21, 'Station 21', 37.7229, -122.4622, 16, 'San Francisco', '2021-09-15'),
  (22, 'Station 22', 37.7173, -122.4656, 20, 'San Francisco', '2021-10-27'),
  (50, 'Station 50', 37.7808, -122.4134, 25, 'San Francisco','2020-02-28');
""")

# Insert sample users
cursor.execute("""
INSERT INTO Service_Requests_311 (unique_key, created_date, closed_date, resolution_action_date, status, status_notes, agency_name, category, complaint_type, incident_address, neighborhood, latitude, longitude, policy_district)
VALUES
    (6078653, '2016-07-13 15:11:07 UTC', '2016-07-14 12:13:22 UTC', '2016-07-14 12:13:22 UTC', 'Closed', 'Field Work has been completed.', 'PUC Sewer Ops', 'Sewer Issues', 'Odor', '5317 GEARY BLVD, SAN FRANCISCO, CA, 94121', 'Outer Richmond', 37.78027, -122.4764, 'RICHMOND'),
    (6078655, '2017-07-15 10:45:12 UTC', '2017-07-15 12:30:00 UTC', '2017-07-15 12:30:00 UTC', 'Closed', 'The issue has been resolved.', 'DPW Street Ops', 'Abandoned Vehicle', 'Abandoned Vehicle', '1665 22ND AVE, SAN FRANCISCO, CA, 94122', 'Sunset/Parkside', 37.7542, -122.4764, 'TARAVAL'),
    (6078656, '2018-07-16 14:20:55 UTC', '2018-07-16 15:35:00 UTC', '2018-07-16 15:35:00 UTC', 'Closed', 'Service completed.', 'DPW Enforcement', 'Illegal Postings', 'Illegal Postings', '1200 9TH AVE, SAN FRANCISCO, CA, 94122', 'Inner Sunset', 37.7646, -122.4665, 'TARAVAL'),
    (6078657, '2019-07-17 08:55:40 UTC', '2019-07-17 10:45:00 UTC', '2019-07-17 10:45:00 UTC', 'Closed', 'The issue has been resolved.', 'DPH Environmental Health', 'Graffiti Public Property', 'Graffiti', '601 ELLIS ST, SAN FRANCISCO, CA, 94109', 'Tenderloin', 37.7856, -122.4179, 'TENDERLOIN'),
    (6078658, '2020-07-18 11:30:20 UTC', '2020-07-18 14:20:00 UTC', '2020-07-18 14:20:00 UTC', 'Closed', 'Service completed.', 'DPW Street Ops', 'Street and Sidewalk Cleaning', 'Human Waste', '655 AUBURN AVE, SAN FRANCISCO, CA, 94112', 'Oceanview/Merced/Ingleside', 37.7229, -122.4479, 'INGLESIDE'),
    (6078659, '2021-07-19 13:15:00 UTC', '2021-07-19 15:00:00 UTC', '2021-07-19 15:00:00 UTC', 'Closed', 'Field Work has been completed.', 'DPW Street Ops', 'Graffiti', 'Graffiti', '3031 16TH ST, SAN FRANCISCO, CA, 94103', 'Mission', 37.7647, -122.4177, 'MISSION'),
    (6078660, '2022-07-20 16:40:35 UTC', '2022-07-20 17:45:00 UTC', '2022-07-20 17:45:00 UTC', 'Closed', 'The issue has been resolved.', 'PUC Sewer Ops', 'Sewer Issues', 'Sewer Issues', '1516 JONES ST, SAN FRANCISCO, CA, 94109', 'Nob Hill', 37.7947, -122.4149, 'NORTHERN'),
    (6078661, '2023-07-21 09:10:25 UTC', '2023-07-21 10:30:00 UTC', '2023-07-21 10:30:00 UTC', 'Closed', 'Service completed.', 'DPW Street Ops', 'Street and Sidewalk Cleaning', 'Encampments', '3850 CALIFORNIA ST, SAN FRANCISCO, CA, 94118', 'Inner Richmond', 37.7871, -122.4528, 'RICHMOND'),
    (6078662, '2024-04-22 12:05:50 UTC', '2024-04-22 13:20:00 UTC', '2024-07-22 13:20:00 UTC', 'Closed', 'The issue has been resolved.', 'DPH Environmental Health', 'Graffiti', 'Graffiti', '2400 MARIPOSA ST, SAN FRANCISCO, CA, 94107', 'Potrero Hill', 37.7639, -122.4061, 'BAYVIEW'),
    (6078663, '2023-07-23 14:55:10 UTC', '2023-07-23 16:00:00 UTC', '2023-07-23 16:00:00 UTC', 'Closed', 'Field Work has been completed.', 'DPW Street Ops', 'Street and Sidewalk Cleaning', 'Encampments', '899 BUSH ST, SAN FRANCISCO, CA, 94109', 'Nob Hill', 37.7896, -122.4137, 'NORTHERN'),
    (6078664, '2022-07-24 10:30:00 UTC', '2022-07-24 11:45:00 UTC', '2022-07-24 11:45:00 UTC', 'Closed', 'Service completed.', 'DPW Street Ops', 'Abandoned Vehicle', 'Abandoned Vehicle', '600 19TH AVE, SAN FRANCISCO, CA, 94121', 'Inner Richmond', 37.7752, -122.4787, 'RICHMOND'),
    (6078665, '2021-07-25 11:40:15 UTC', '2021-07-25 12:55:00 UTC', '2021-07-25 12:55:00 UTC', 'Closed', 'The issue has been resolved.', 'PUC Sewer Ops', 'Sewer Issues', 'Sewer Issues', '165 JONES ST, SAN FRANCISCO, CA, 94102', 'Tenderloin', 37.7809, -122.4127, 'TENDERLOIN'),
    (6078666, '2020-07-26 15:00:20 UTC', '2020-07-26 16:15:00 UTC', '2020-07-26 16:15:00 UTC', 'Closed', 'Service completed.', 'DPW Enforcement', 'Illegal Postings', 'Illegal Postings', '3398 26TH ST, SAN FRANCISCO, CA, 94110', 'Mission', 37.7484, -122.4174, 'MISSION'),
    (6078667, '2019-07-27 08:35:40 UTC', '2019-07-27 10:30:00 UTC', '2019-07-27 10:30:00 UTC', 'Closed', 'The issue has been resolved.', 'DPH Environmental Health', 'Graffiti Public Property', 'Graffiti', '785 CLAY ST, SAN FRANCISCO, CA, 94108', 'Chinatown', 37.7921, -122.4094, 'CENTRAL'),
    (6078668, '2018-07-28 10:20:00 UTC', '2018-07-28 11:35:00 UTC', '2018-07-28 11:35:00 UTC', 'Closed', 'Service completed.', 'DPW Street Ops', 'Street and Sidewalk Cleaning', 'Human Waste', '400 HYDE ST, SAN FRANCISCO, CA, 94109', 'Tenderloin', 37.7843, -122.4154, 'TENDERLOIN'),
    (6078669, '2017-07-29 14:45:35 UTC', '2017-07-29 16:00:00 UTC', '2017-07-29 16:00:00 UTC', 'Closed', 'Field Work has been completed.', 'DPH Environmental Health', 'Graffiti', 'Graffiti', '2988 20TH ST, SAN FRANCISCO, CA, 94110', 'Mission', 37.7587, -122.4112, 'MISSION'),
    (6078670, '2016-07-30 17:00:50 UTC', '2016-07-30 18:15:00 UTC', '2016-07-30 18:15:00 UTC', 'Closed', 'The issue has been resolved.', 'PUC Sewer Ops', 'Sewer Issues', 'Sewer Issues', '945 GEARY ST, SAN FRANCISCO, CA, 94109', 'Tenderloin', 37.7862, -122.4173, 'TENDERLOIN'),
    (6078671, '2016-07-31 09:10:00 UTC', '2016-07-31 10:25:00 UTC', '2016-07-31 10:25:00 UTC', 'Closed', 'Service completed.', 'DPW Street Ops', 'Street and Sidewalk Cleaning', 'Encampments', '1100 TURK ST, SAN FRANCISCO, CA, 94102', 'Western Addition', 37.7804, -122.4293, 'NORTHERN'),
    (6078672, '2024-04-01 12:35:20 UTC', '2024-04-01 13:45:00 UTC', '2024-08-01 13:45:00 UTC', 'Closed', 'The issue has been resolved.', 'DPH Environmental Health', 'Graffiti', 'Graffiti', '4451 MISSION ST, SAN FRANCISCO, CA, 94112', 'Outer Mission', 37.7235, -122.4353, 'INGLESIDE'),
    (6078673, '2023-08-02 15:50:00 UTC', '2023-08-02 17:00:00 UTC', '2023-08-02 17:00:00 UTC', 'Closed', 'Field Work has been completed.', 'DPW Street Ops', 'Abandoned Vehicle', 'Abandoned Vehicle', '2200 MARKET ST, SAN FRANCISCO, CA, 94114', 'Castro/Upper Market', 37.7661, -122.4321, 'MISSION'),
    (6078674, '2022-08-03 08:10:30 UTC', '2022-08-03 09:25:00 UTC', '2022-08-03 09:25:00 UTC', 'Closed', 'Service completed.', 'DPW Enforcement', 'Illegal Postings', 'Illegal Postings', '2398 POLK ST, SAN FRANCISCO, CA, 94109', 'Russian Hill', 37.7995, -122.4224, 'NORTHERN'),
    (6078675, '2021-08-04 10:45:20 UTC', '2021-08-04 12:00:00 UTC', '2021-08-04 12:00:00 UTC', 'Closed', 'The issue has been resolved.', 'PUC Sewer Ops', 'Sewer Issues', 'Sewer Issues', '750 PINE ST, SAN FRANCISCO, CA, 94108', 'Tenderloin', 37.7899, -122.4091, 'TENDERLOIN');
""")

cursor.execute("""
INSERT INTO Bikeshare_Status(station_id, bikes_available, docks_available, time)
VALUES
  (2, 0, 27, '2016-05-03 14:44:01 UTC'),
  (2, 0, 26, '2016-08-30 12:53:01 UTC'),
  (1, 10, 5, '2020-01-01 08:00:00 UTC'),
  (2, 8, 7,  '2020-01-05 10:00:00 UTC'),
  (3, 15, 5, '2020-03-15 12:00:00 UTC'),
  (4, 7, 11, '2020-04-02 14:00:00 UTC'),
  (5, 20, 2, '2020-05-10 16:00:00 UTC'),
  (6, 12, 7, '2020-06-20 18:00:00 UTC'),
  (7, 18, 3, '2020-07-01 20:00:00 UTC'),
  (8, 5, 12,'2020-08-14 22:00:00 UTC'),
  (9, 4, 12, '2020-09-25 00:00:00 UTC'),
  (10, 9, 11, '2020-10-05 02:00:00 UTC'),
  (11, 11, 7, '2020-11-18 04:00:00 UTC'),
  (12, 16, 7, '2020-12-30 06:00:00 UTC'),
  (13, 3, 11, '2021-01-11 08:00:00 UTC'),
  (14, 8, 8, '2021-02-22 10:00:00 UTC'),
  (15, 13, 6, '2021-03-05 12:00:00 UTC'),
  (16, 17, 4, '2021-04-17 14:00:00 UTC'),
  (17, 9, 11, '2021-05-29 16:00:00 UTC'),
  (18, 6, 12, '2021-06-10 18:00:00 UTC'),
  (19, 2, 15, '2021-07-22 20:00:00 UTC'),
  (20, 11, 4, '2021-08-04 22:00:00 UTC'),
  (21, 14, 4, '2021-09-15 00:00:00 UTC'),
  (22, 19, 1, '2021-10-27 02:00:00 UTC'),
  (50, 22, 3, '2020-02-28 04:00:00 UTC');

""")

cursor.execute("""
INSERT INTO Bikeshare_Trips(trip_id, duration_sec, start_date, start_station_name, start_station_id, end_date, end_station_name, end_station_id, bike_number, zip_code)
VALUES
    (1235850, 1540, '2016-06-11 08:19:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2016-06-11 08:45:00 UTC', 'San Jose Diridon Caltrain Station', 2, 124, '15206'),
    (279046, 4338, '2014-05-10 11:20:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-05-10 12:32:00 UTC', 'San Jose Diridon Caltrain Station', 2, 713, 95126),
    (55603, 1856, '2013-10-12 15:15:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2013-10-12 15:46:00 UTC', 'San Jose Diridon Caltrain Station', 2, 52, 94305),
    (227988, 150, '2014-03-26 19:16:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-03-26 19:18:00 UTC', 'San Jose Diridon Caltrain Station', 2, 184, 90036),
    (123925, 710, '2013-12-13 06:23:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2013-12-13 06:35:00 UTC', 'San Jose Diridon Caltrain Station', 2, 297, 95124),
    (933220, 1596, '2015-09-16 09:59:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2015-09-16 10:25:00 UTC', 'San Jose Diridon Caltrain Station', 2, 648, 11217),
    (900284, 31298, '2015-08-21 15:38:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2015-08-22 00:20:00 UTC', 'San Jose Diridon Caltrain Station', 2, 38, 95126),
    (28818, 2283, '2013-09-20 13:22:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2013-09-20 14:00:00 UTC', 'San Jose Diridon Caltrain Station', 2, 23, 95054),
    (142985, 4106, '2014-01-05 14:04:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-01-05 15:13:00 UTC', 'San Jose Diridon Caltrain Station', 2, 211, 8102),
    (515231, 8707, '2014-10-25 14:39:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-10-25 17:04:00 UTC', 'San Jose Diridon Caltrain Station', 2, 131, 95060),
    (859939, 203, '2015-07-23 20:21:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2015-07-23 20:25:00 UTC', 'San Jose Diridon Caltrain Station', 2, 247, 11210),
    (462893, 7485, '2014-09-21 18:38:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-09-21 20:43:00 UTC', 'San Jose Diridon Caltrain Station', 2, 162, 95076),
    (155588, 3712, '2014-01-16 10:17:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-01-16 11:18:00 UTC', 'San Jose Diridon Caltrain Station', 2, 78, 94106),
    (712140, 33084, '2015-04-04 12:48:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2015-04-04 22:00:00 UTC', 'San Jose Diridon Caltrain Station', 2, 31, 95037),
    (483498, 4791, '2014-10-05 09:56:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-10-05 11:15:00 UTC', 'San Jose Diridon Caltrain Station', 2, 186, 95119),
    (254478, 4878, '2014-04-19 15:17:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-04-19 16:38:00 UTC', 'San Jose Diridon Caltrain Station', 2, 101, 94305),
    (720010, 11107, '2015-04-10 09:32:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2015-04-10 12:37:00 UTC', 'San Jose Diridon Caltrain Station', 2, 81, 3045),
    (153060, 110, '2014-01-14 16:12:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-01-14 16:14:00 UTC', 'San Jose Diridon Caltrain Station', 2, 108, 38478),
    (238186, 4397, '2014-04-06 15:06:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-04-06 16:19:00 UTC', 'San Jose Diridon Caltrain Station', 2, 70, 95020),
    (779245, 2876, '2015-05-23 14:46:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2015-05-23 15:34:00 UTC', 'San Jose Diridon Caltrain Station', 2, 143, 98059),
    (142983, 4121, '2014-01-05 14:04:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-01-05 15:13:00 UTC', 'San Jose Diridon Caltrain Station', 2, 60, 8504),
    (1100028, 787, '2016-02-21 16:15:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2016-02-21 16:28:00 UTC', 'San Jose Diridon Caltrain Station', 2, 60, 95020),
    (65470, 3711, '2013-10-20 12:04:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2013-10-20 13:06:00 UTC', 'San Jose Diridon Caltrain Station', 2, 64, 95116),
    (1323137, 631, '2016-08-19 13:29:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2016-08-19 13:39:00 UTC', 'San Jose Diridon Caltrain Station', 2, 102, 95136),
    (1236213, 1463, '2016-06-11 19:54:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2016-06-11 20:18:00 UTC', 'San Jose Diridon Caltrain Station', 2, 124, 15206),
    (11850, 513, '2013-09-05 18:42:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2013-09-05 18:50:00 UTC', 'San Jose Diridon Caltrain Station', 2, 641, 95122),
    (452207, 72687, '2014-09-14 15:03:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-09-15 11:15:00 UTC', 'San Jose Diridon Caltrain Station', 2, 194, 81),
    (142981, 4144, '2014-01-05 14:04:00 UTC', 'San Jose Diridon Caltrain Station', 2, '2014-01-05 15:13:00 UTC', 'San Jose Diridon Caltrain Station', 2, 145, 8102);
""")

cursor.execute("""
INSERT INTO Film_Locations (title, release_year, locations, production_company, director, writer, actor_1, actor_2)
VALUES
    ('The Towering Inferno', 1974, 'Bank of America Building (555 California Street, Financial District)', 'Irwin Allen Productions', 'John Guillermin', 'Stirling Silliphant', 'Steve McQueen', 'Paul Newman'),
    ('The Towering Inferno', 1974, 'Fairmont Hotel (950 Mason Street, Nob Hill)', 'Irwin Allen Productions', 'John Guillermin', 'Stirling Silliphant', 'Steve McQueen', 'Paul Newman'),
    ('The Towering Inferno', 1974, 'Grace Cathedral Episcopal Church (1100 California Street)', 'Irwin Allen Productions', 'John Guillermin', 'Stirling Silliphant', 'Steve McQueen', 'Paul Newman'),
    ('The Towering Inferno', 1974, 'Firestation #38 (California & Laguna)', 'Irwin Allen Productions', 'John Guillermin', 'Stirling Silliphant', 'Steve McQueen', 'Paul Newman'),
    ('The Towering Inferno', 1974, 'Hyatt Regency Hotel (5 Embarcadero Center, Financial District)', 'Irwin Allen Productions', 'John Guillermin', 'Stirling Silliphant', 'Steve McQueen', 'Paul Newman'),
    ('The Towering Inferno', 1974, 'San Francisco Fire Station 38 (2150 California Street, Pacific Heights)', 'Irwin Allen Productions', 'John Guillermin', 'Stirling Silliphant', 'Steve McQueen', 'Paul Newman'),
    ('Faces of Death', 1978, 'City Lights Bookstore', 'F.O.D Productions', 'Conan Le Cilaire', 'Alan Black', 'Michael Carr', 'Paul Newman'),
    ('Attack of the Killer Tomatoes', 1978, 'Hyde Street Cable Car', 'Four Square Productions', 'John DeBello', 'John DeBello', 'Steve McQueen', 'Paul Newman'),
    ('Heart Beat', 1980, 'Washington Square Bar & Grill (1707 Powell)', 'Orion Pictures Corporation', 'John Byrum', 'John Byrum', 'Nick Nolte', 'Sissy Spacek'),
    ('Heart Beat', 1980, 'City Lights Bookstore (261 Columbus)', 'Orion Pictures Corporation', 'John Byrum', 'John Byrum', 'Nick Nolte', 'Sissy Spacek');
""")

cursor.execute("""
INSERT INTO SFPD_Incidents (unique_key, category, descript, dateofweek, pddistrict, resolution, address, latitude, longitude, timestamp)
VALUES
    (123456, 'LARCENY/THEFT', 'PETTY THEFT OF PROPERTY', 'Monday', 'CENTRAL', 'NONE', '123 Main St', 37.7749, -122.4194, '2024-04-28 10:15:00'),
    (234567, 'BURGLARY', 'FORCIBLE ENTRY', 'Tuesday', 'SOUTHERN', 'ARREST, BOOKED', '456 Elm St', 37.7822, -122.4057, '2024-04-28 12:30:00'),
    (345678, 'VANDALISM', 'MALICIOUS MISCHIEF, VANDALISM', 'Wednesday', 'MISSION', 'NONE', '789 Oak St', 37.7577, -122.4376, '2024-04-28 14:45:00'),
    (456789, 'ASSAULT', 'BATTERY', 'Thursday', 'NORTHERN', 'ARREST, CITED', '101 Pine St', 37.7915, -122.4072, '2024-04-28 16:00:00'),
    (567890, 'DRUG/NARCOTIC', 'POSSESSION OF MARIJUANA', 'Friday', 'TENDERLOIN', 'NONE', '202 Market St', 37.7858, -122.4062, '2024-04-28 18:15:00'),
    (678901, 'ROBBERY', 'ROBBERY, BODILY FORCE', 'Saturday', 'RICHMOND', 'NONE', '303 Broadway St', 37.7986, -122.4063, '2024-04-28 20:30:00'),
    (789012, 'WEAPON LAWS', 'POSSESSION OF FIREARM', 'Sunday', 'TARAVAL', 'ARREST, BOOKED', '404 Howard St', 37.7875, -122.3984, '2024-04-29 09:45:00'),
    (890123, 'PROSTITUTION', 'SOLICITS FOR ACT OF PROSTITUTION', 'Monday', 'INGLESIDE', 'NONE', '505 Washington St', 37.7936, -122.3964, '2024-04-29 11:00:00'),
    (901234, 'DRUNKENNESS', 'DRUNKENNESS', 'Tuesday', 'BAYVIEW', 'NONE', '606 Powell St', 37.7849, -122.4078, '2024-04-29 13:15:00'),
    (912345, 'TRESPASS', 'TRESPASSING', 'Wednesday', 'CENTRAL', 'NONE', '707 Sacramento St', 37.7937, -122.4031, '2024-04-29 15:30:00'),
    (923456, 'DISORDERLY CONDUCT', 'DISTURBING THE PEACE', 'Thursday', 'SOUTHERN', 'NONE', '808 Sansome St', 37.7973, -122.4018, '2024-04-29 17:45:00'),
    (934567, 'SECONDARY CODES', 'SUSPICIOUS OCCURRENCE', 'Friday', 'MISSION', 'NONE', '909 Clay St', 37.7926, -122.4042, '2024-04-29 19:00:00'),
    (945678, 'FRAUD', 'CREDIT CARD, THEFT OF ACCESS CARD', 'Saturday', 'NORTHERN', 'NONE', '1010 Market St', 37.7867, -122.4062, '2024-04-29 21:15:00'),
    (956789, 'RUNAWAY', 'RUNAWAY', 'Sunday', 'TENDERLOIN', 'NONE', '1111 Battery St', 37.8062, -122.4018, '2024-04-30 10:30:00'),
    (967890, 'BRIBERY', 'BRIBERY', 'Monday', 'RICHMOND', 'NONE', '1212 Montgomery St', 37.7915, -122.4038, '2024-04-30 12:45:00'),
    (978901, 'EMBEZZLEMENT', 'EMBEZZLEMENT, GRAND THEFT BY EMPLOYEE', 'Tuesday', 'TARAVAL', 'NONE', '1313 Pine St', 37.7919, -122.4114, '2024-04-30 15:00:00'),
    (989012, 'SUICIDE', 'SUICIDE', 'Wednesday', 'INGLESIDE', 'NONE', '1414 Post St', 37.7871, -122.4143, '2024-04-30 17:15:00'),
    (990123, 'EXTORTION', 'EXTORTION', 'Thursday', 'BAYVIEW', 'NONE', '1515 Bush St', 37.7889, -122.4322, '2024-04-30 19:30:00'),
    (901234, 'FAMILY OFFENSES', 'NON-CRIMINAL (FAMILY OFFENSES)', 'Friday', 'CENTRAL', 'NONE', '1616 Jones St', 37.7955, -122.4153, '2024-04-30 21:45:00'),
    (912345, 'SEX OFFENSES, FORCIBLE', 'RAPE, FORCIBLE', 'Saturday', 'SOUTHERN', 'NONE', '1717 Hyde St', 37.7952, -122.4186, '2024-05-01 09:00:00')
""")


cursor.execute("""
INSERT INTO SFFD_Service_Calls (call_number, unit_id, incident_number, call_type, call_date, response_timestamp, on_scene_timestamp, address, city, zipcode_of_incident, station_area)
VALUES
    (123456, 'E20', 789012, 'Fire', '2024-04-23', '2024-04-23 08:00:00', '2024-04-23 08:15:00', '123 Main St', 'San Francisco', '94102', 'Station 1'),
    (234567, 'E10', 890123, 'Medical', '2024-04-23', '2024-04-23 09:30:00', '2024-04-23 09:45:00', '456 Oak St', 'San Francisco', '94110', 'Station 5'),
    (345678, 'E15', 901234, 'Rescue', '2024-04-23', '2024-04-23 11:00:00', '2024-04-23 11:20:00', '789 Elm St', 'San Francisco', '94122', 'Station 8'),
    (456789, 'E25', 912345, 'Fire', '2024-04-23', '2024-04-23 13:15:00', '2024-04-23 13:30:00', '321 Pine St', 'San Francisco', '94117', 'Station 4'),
    (567890, 'E30', 923456, 'Medical', '2024-04-23', '2024-04-23 15:45:00', '2024-04-23 16:00:00', '654 Cedar St', 'San Francisco', '94103', 'Station 2'),
    (678901, 'E18', 934567, 'Fire', '2024-04-24', '2024-04-24 08:30:00', '2024-04-24 08:45:00', '789 Maple St', 'San Francisco', '94109', 'Station 3'),
    (789012, 'E5', 945678, 'Medical', '2024-04-24', '2024-04-24 10:00:00', '2024-04-24 10:15:00', '456 Walnut St', 'San Francisco', '94115', 'Station 6'),
    (890123, 'E17', 956789, 'Rescue', '2024-04-24', '2024-04-24 11:45:00', '2024-04-24 12:00:00', '123 Pineapple St', 'San Francisco', '94107', 'Station 9'),
    (901234, 'E3', 967890, 'Fire', '2024-04-24', '2024-04-24 13:30:00', '2024-04-24 13:45:00', '789 Orange St', 'San Francisco', '94112', 'Station 7'),
    (912345, 'E12', 978901, 'Medical', '2024-04-24', '2024-04-24 15:00:00', '2024-04-24 15:15:00', '456 Cherry St', 'San Francisco', '94121', 'Station 10'),
    (923456, 'E8', 989012, 'Rescue', '2024-04-24', '2024-04-24 16:45:00', '2024-04-24 17:00:00', '123 Lemon St', 'San Francisco', '94118', 'Station 12'),
    (934567, 'E21', 990123, 'Fire', '2024-04-25', '2024-04-25 09:15:00', '2024-04-25 09:30:00', '789 Peach St', 'San Francisco', '94133', 'Station 11'),
    (945678, 'E6', 901234, 'Medical', '2024-04-25', '2024-04-25 10:45:00', '2024-04-25 11:00:00', '456 Plum St', 'San Francisco', '94123', 'Station 14'),
    (956789, 'E16', 912345, 'Rescue', '2024-04-25', '2024-04-25 12:30:00', '2024-04-25 12:45:00', '123 Pear St', 'San Francisco', '94131', 'Station 16'),
    (967890, 'E7', 923456, 'Fire', '2024-04-25', '2024-04-25 14:00:00', '2024-04-25 14:15:00', '789 Banana St', 'San Francisco', '94127', 'Station 13'),
    (978901, 'E22', 934567, 'Medical', '2024-04-25', '2024-04-25 15:30:00', '2024-04-25 15:45:00', '456 Kiwi St', 'San Francisco', '94132', 'Station 15'),
    (989012, 'E9', 945678, 'Rescue', '2024-04-25', '2024-04-25 17:00:00', '2024-04-25 17:15:00', '123 Grape St', 'San Francisco', '94116', 'Station 17'),
    (990123, 'E19', 956789, 'Fire', '2024-04-26', '2024-04-26 08:45:00', '2024-04-26 09:00:00', '789 Mango St', 'San Francisco', '94124', 'Station 18'),
    (901234, 'E4', 967890, 'Medical', '2024-04-26', '2024-04-26 10:15:00', '2024-04-26 10:30:00', '456 Papaya St', 'San Francisco', '94134', 'Station 20'),
    (912345, 'E14', 978901, 'Rescue', '2024-04-26', '2024-04-26 11:45:00', '2024-04-26 12:00:00', '123 Coconut St', 'San Francisco', '94105', 'Station 19'),
    (923456, 'E11', 989012, 'Fire', '2024-04-26', '2024-04-26 13:30:00', '2024-04-26 13:45:00', '789 Guava St', 'San Francisco', '94130', 'Station 21'),
    (934567, 'E23', 990123, 'Medical', '2024-04-26', '2024-04-26 15:00:00', '2024-04-26 15:15:00', '456 Dragonfruit St', 'San Francisco', '94158', 'Station 22'),
    (945678, 'E2', 901234, 'Rescue', '2024-04-26', '2024-04-26 16:30:00', '2024-04-26 16:45:00', '123 Lime St', 'San Francisco', '94129', 'Station 24'),
    (956789, 'E20', 912345, 'Fire', '2024-04-26', '2024-04-26 18:00:00', '2024-04-26 18:15:00', '789 Pineapple St', 'San Francisco', '94111', 'Station 23'),
    (967890, 'E10', 923456, 'Medical', '2024-04-27', '2024-04-27 09:30:00', '2024-04-27 09:45:00', '456 Persimmon St', 'San Francisco', '94104', 'Station 26');

""")

# Commit changes and close the connection
conn.commit()
conn.close()

runSql('Service_Requests_311', "select * from Service_Requests_311;")
runSql('Bikeshare_Stations', "select * from Bikeshare_Stations;")
runSql('Bikeshare_Status', "select * from Bikeshare_Status;")
runSql('Bikeshare_Trips', "select * from Bikeshare_Trips;")
runSql('Film_Locations', "select * from Film_Locations;")
runSql('SFPD_Incidents', "select * from SFPD_Incidents;")
runSql('SFFD_Service_Calls', "select * from SFFD_Service_Calls;")

unique_key,created_date,closed_date,resolution_action_date,status,status_notes,agency_name,category,complaint_type,incident_address,neighborhood,latitude,longitude,policy_district
6078653,2016-07-13 15:11:07 UTC,2016-07-14 12:13:22 UTC,2016-07-14 12:13:22 UTC,Closed,Field Work has been completed.,PUC Sewer Ops,Sewer Issues,Odor,"5317 GEARY BLVD, SAN FRANCISCO, CA, 94121",Outer Richmond,37.78027,-122.4764,RICHMOND
6078655,2017-07-15 10:45:12 UTC,2017-07-15 12:30:00 UTC,2017-07-15 12:30:00 UTC,Closed,The issue has been resolved.,DPW Street Ops,Abandoned Vehicle,Abandoned Vehicle,"1665 22ND AVE, SAN FRANCISCO, CA, 94122",Sunset/Parkside,37.75420,-122.4764,TARAVAL
6078656,2018-07-16 14:20:55 UTC,2018-07-16 15:35:00 UTC,2018-07-16 15:35:00 UTC,Closed,Service completed.,DPW Enforcement,Illegal Postings,Illegal Postings,"1200 9TH AVE, SAN FRANCISCO, CA, 94122",Inner Sunset,37.76460,-122.4665,TARAVAL
6078657,2019-07-17 08:55:40 UTC,2019-07-17 10:45:00 UTC,2019-07-17 10:45:00 UTC,Closed,The issue has been resolved.,DPH Environmental Health,Graffiti Public Property,Graffiti,"601 ELLIS ST, SAN FRANCISCO, CA, 94109",Tenderloin,37.78560,-122.4179,TENDERLOIN
6078658,2020-07-18 11:30:20 UTC,2020-07-18 14:20:00 UTC,2020-07-18 14:20:00 UTC,Closed,Service completed.,DPW Street Ops,Street and Sidewalk Cleaning,Human Waste,"655 AUBURN AVE, SAN FRANCISCO, CA, 94112",Oceanview/Merced/Ingleside,37.72290,-122.4479,INGLESIDE
6078659,2021-07-19 13:15:00 UTC,2021-07-19 15:00:00 UTC,2021-07-19 15:00:00 UTC,Closed,Field Work has been completed.,DPW Street Ops,Graffiti,Graffiti,"3031 16TH ST, SAN FRANCISCO, CA, 94103",Mission,37.76470,-122.4177,MISSION
6078660,2022-07-20 16:40:35 UTC,2022-07-20 17:45:00 UTC,2022-07-20 17:45:00 UTC,Closed,The issue has been resolved.,PUC Sewer Ops,Sewer Issues,Sewer Issues,"1516 JONES ST, SAN FRANCISCO, CA, 94109",Nob Hill,37.79470,-122.4149,NORTHERN
6078661,2023-07-21 09:10:25 UTC,2023-07-21 10:30:00 UTC,2023-07-21 10:30:00 UTC,Closed,Service completed.,DPW Street Ops,Street and Sidewalk Cleaning,Encampments,"3850 CALIFORNIA ST, SAN FRANCISCO, CA, 94118",Inner Richmond,37.78710,-122.4528,RICHMOND
6078662,2024-04-22 12:05:50 UTC,2024-04-22 13:20:00 UTC,2024-07-22 13:20:00 UTC,Closed,The issue has been resolved.,DPH Environmental Health,Graffiti,Graffiti,"2400 MARIPOSA ST, SAN FRANCISCO, CA, 94107",Potrero Hill,37.76390,-122.4061,BAYVIEW
6078663,2023-07-23 14:55:10 UTC,2023-07-23 16:00:00 UTC,2023-07-23 16:00:00 UTC,Closed,Field Work has been completed.,DPW Street Ops,Street and Sidewalk Cleaning,Encampments,"899 BUSH ST, SAN FRANCISCO, CA, 94109",Nob Hill,37.78960,-122.4137,NORTHERN


station_id,name,latitude,longitude,dockcount,landmark,installation_date
1,Japantown,37.348742,-121.894715,15,San Jose,2020-01-01
2,Station 2,37.776600,-122.395600,15,San Francisco,2020-01-05
3,Station 3,37.774900,-122.419400,20,San Francisco,2020-03-15
4,Station 4,37.775500,-122.404500,18,San Francisco,2020-04-02
5,Station 5,37.783200,-122.398600,22,San Francisco,2020-05-10
6,Station 6,37.791000,-122.401600,19,San Francisco,2020-06-20
7,Station 7,37.794800,-122.404100,21,San Francisco,2020-07-01
8,Station 8,37.789200,-122.409900,17,San Francisco,2020-08-14
9,Station 9,37.785600,-122.414300,16,San Francisco,2020-09-25
10,Station 10,37.781400,-122.421900,20,San Francisco,2020-10-05


station_id,bikes_available,docks_available,time
2,0,27,2016-05-03 14:44:01 UTC
2,0,26,2016-08-30 12:53:01 UTC
1,10,5,2020-01-01 08:00:00 UTC
2,8,7,2020-01-05 10:00:00 UTC
3,15,5,2020-03-15 12:00:00 UTC
4,7,11,2020-04-02 14:00:00 UTC
5,20,2,2020-05-10 16:00:00 UTC
6,12,7,2020-06-20 18:00:00 UTC
7,18,3,2020-07-01 20:00:00 UTC
8,5,12,2020-08-14 22:00:00 UTC


trip_id,duration_sec,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_number,zip_code
1235850,1540,2016-06-11 08:19:00 UTC,San Jose Diridon Caltrain Station,2,2016-06-11 08:45:00 UTC,San Jose Diridon Caltrain Station,2,124,15206
279046,4338,2014-05-10 11:20:00 UTC,San Jose Diridon Caltrain Station,2,2014-05-10 12:32:00 UTC,San Jose Diridon Caltrain Station,2,713,95126
55603,1856,2013-10-12 15:15:00 UTC,San Jose Diridon Caltrain Station,2,2013-10-12 15:46:00 UTC,San Jose Diridon Caltrain Station,2,52,94305
227988,150,2014-03-26 19:16:00 UTC,San Jose Diridon Caltrain Station,2,2014-03-26 19:18:00 UTC,San Jose Diridon Caltrain Station,2,184,90036
123925,710,2013-12-13 06:23:00 UTC,San Jose Diridon Caltrain Station,2,2013-12-13 06:35:00 UTC,San Jose Diridon Caltrain Station,2,297,95124
933220,1596,2015-09-16 09:59:00 UTC,San Jose Diridon Caltrain Station,2,2015-09-16 10:25:00 UTC,San Jose Diridon Caltrain Station,2,648,11217
900284,31298,2015-08-21 15:38:00 UTC,San Jose Diridon Caltrain Station,2,2015-08-22 00:20:00 UTC,San Jose Diridon Caltrain Station,2,38,95126
28818,2283,2013-09-20 13:22:00 UTC,San Jose Diridon Caltrain Station,2,2013-09-20 14:00:00 UTC,San Jose Diridon Caltrain Station,2,23,95054
142985,4106,2014-01-05 14:04:00 UTC,San Jose Diridon Caltrain Station,2,2014-01-05 15:13:00 UTC,San Jose Diridon Caltrain Station,2,211,8102
515231,8707,2014-10-25 14:39:00 UTC,San Jose Diridon Caltrain Station,2,2014-10-25 17:04:00 UTC,San Jose Diridon Caltrain Station,2,131,95060


title,release_year,locations,production_company,director,writer,actor_1,actor_2
The Towering Inferno,1974,"Bank of America Building (555 California Street, Financial District)",Irwin Allen Productions,John Guillermin,Stirling Silliphant,Steve McQueen,Paul Newman
The Towering Inferno,1974,"Fairmont Hotel (950 Mason Street, Nob Hill)",Irwin Allen Productions,John Guillermin,Stirling Silliphant,Steve McQueen,Paul Newman
The Towering Inferno,1974,Grace Cathedral Episcopal Church (1100 California Street),Irwin Allen Productions,John Guillermin,Stirling Silliphant,Steve McQueen,Paul Newman
The Towering Inferno,1974,Firestation #38 (California & Laguna),Irwin Allen Productions,John Guillermin,Stirling Silliphant,Steve McQueen,Paul Newman
The Towering Inferno,1974,"Hyatt Regency Hotel (5 Embarcadero Center, Financial District)",Irwin Allen Productions,John Guillermin,Stirling Silliphant,Steve McQueen,Paul Newman
The Towering Inferno,1974,"San Francisco Fire Station 38 (2150 California Street, Pacific Heights)",Irwin Allen Productions,John Guillermin,Stirling Silliphant,Steve McQueen,Paul Newman
Faces of Death,1978,City Lights Bookstore,F.O.D Productions,Conan Le Cilaire,Alan Black,Michael Carr,Paul Newman
Attack of the Killer Tomatoes,1978,Hyde Street Cable Car,Four Square Productions,John DeBello,John DeBello,Steve McQueen,Paul Newman
Heart Beat,1980,Washington Square Bar & Grill (1707 Powell),Orion Pictures Corporation,John Byrum,John Byrum,Nick Nolte,Sissy Spacek
Heart Beat,1980,City Lights Bookstore (261 Columbus),Orion Pictures Corporation,John Byrum,John Byrum,Nick Nolte,Sissy Spacek


unique_key,category,descript,dateofweek,pddistrict,resolution,address,latitude,longitude,timestamp
123456,LARCENY/THEFT,PETTY THEFT OF PROPERTY,Monday,CENTRAL,NONE,123 Main St,37.7749,-122.4194,2024-04-28 10:15:00
234567,BURGLARY,FORCIBLE ENTRY,Tuesday,SOUTHERN,"ARREST, BOOKED",456 Elm St,37.7822,-122.4057,2024-04-28 12:30:00
345678,VANDALISM,"MALICIOUS MISCHIEF, VANDALISM",Wednesday,MISSION,NONE,789 Oak St,37.7577,-122.4376,2024-04-28 14:45:00
456789,ASSAULT,BATTERY,Thursday,NORTHERN,"ARREST, CITED",101 Pine St,37.7915,-122.4072,2024-04-28 16:00:00
567890,DRUG/NARCOTIC,POSSESSION OF MARIJUANA,Friday,TENDERLOIN,NONE,202 Market St,37.7858,-122.4062,2024-04-28 18:15:00
678901,ROBBERY,"ROBBERY, BODILY FORCE",Saturday,RICHMOND,NONE,303 Broadway St,37.7986,-122.4063,2024-04-28 20:30:00
789012,WEAPON LAWS,POSSESSION OF FIREARM,Sunday,TARAVAL,"ARREST, BOOKED",404 Howard St,37.7875,-122.3984,2024-04-29 09:45:00
890123,PROSTITUTION,SOLICITS FOR ACT OF PROSTITUTION,Monday,INGLESIDE,NONE,505 Washington St,37.7936,-122.3964,2024-04-29 11:00:00
901234,DRUNKENNESS,DRUNKENNESS,Tuesday,BAYVIEW,NONE,606 Powell St,37.7849,-122.4078,2024-04-29 13:15:00
912345,TRESPASS,TRESPASSING,Wednesday,CENTRAL,NONE,707 Sacramento St,37.7937,-122.4031,2024-04-29 15:30:00


call_number,unit_id,incident_number,call_type,call_date,response_timestamp,on_scene_timestamp,address,city,zipcode_of_incident,station_area
123456,E20,789012,Fire,2024-04-23,2024-04-23 08:00:00,2024-04-23 08:15:00,123 Main St,San Francisco,94102,Station 1
234567,E10,890123,Medical,2024-04-23,2024-04-23 09:30:00,2024-04-23 09:45:00,456 Oak St,San Francisco,94110,Station 5
345678,E15,901234,Rescue,2024-04-23,2024-04-23 11:00:00,2024-04-23 11:20:00,789 Elm St,San Francisco,94122,Station 8
456789,E25,912345,Fire,2024-04-23,2024-04-23 13:15:00,2024-04-23 13:30:00,321 Pine St,San Francisco,94117,Station 4
567890,E30,923456,Medical,2024-04-23,2024-04-23 15:45:00,2024-04-23 16:00:00,654 Cedar St,San Francisco,94103,Station 2
678901,E18,934567,Fire,2024-04-24,2024-04-24 08:30:00,2024-04-24 08:45:00,789 Maple St,San Francisco,94109,Station 3
789012,E5,945678,Medical,2024-04-24,2024-04-24 10:00:00,2024-04-24 10:15:00,456 Walnut St,San Francisco,94115,Station 6
890123,E17,956789,Rescue,2024-04-24,2024-04-24 11:45:00,2024-04-24 12:00:00,123 Pineapple St,San Francisco,94107,Station 9
901234,E3,967890,Fire,2024-04-24,2024-04-24 13:30:00,2024-04-24 13:45:00,789 Orange St,San Francisco,94112,Station 7
912345,E12,978901,Medical,2024-04-24,2024-04-24 15:00:00,2024-04-24 15:15:00,456 Cherry St,San Francisco,94121,Station 10


In [19]:
# run query
qry_most_common_311_service_calls = """
SELECT
    incident_address, category,COUNT(*) AS num_incidents
FROM
    Service_Requests_311
GROUP BY
    category
ORDER BY
    num_incidents DESC;
"""
runSql('Most common 311 Service Calls based on category and location', qry_most_common_311_service_calls)

incident_address,category,num_incidents
"655 AUBURN AVE, SAN FRANCISCO, CA, 94112",Street and Sidewalk Cleaning,5
"5317 GEARY BLVD, SAN FRANCISCO, CA, 94121",Sewer Issues,5
"3031 16TH ST, SAN FRANCISCO, CA, 94103",Graffiti,4
"1200 9TH AVE, SAN FRANCISCO, CA, 94122",Illegal Postings,3
"1665 22ND AVE, SAN FRANCISCO, CA, 94122",Abandoned Vehicle,3
"601 ELLIS ST, SAN FRANCISCO, CA, 94109",Graffiti Public Property,2


In [20]:
qry_most_docks_available_station = """
SELECT *
FROM Bikeshare_Stations
ORDER BY dockcount DESC;
"""
runSql('Most docks availble station', qry_most_docks_available_station)

station_id,name,latitude,longitude,dockcount,landmark,installation_date
50,Station 50,37.780800,-122.413400,25,San Francisco,2020-02-28
12,Station 12,37.773300,-122.431700,23,San Francisco,2020-12-30
5,Station 5,37.783200,-122.398600,22,San Francisco,2020-05-10
7,Station 7,37.794800,-122.404100,21,San Francisco,2020-07-01
16,Station 16,37.750900,-122.445200,21,San Francisco,2021-04-17
3,Station 3,37.774900,-122.419400,20,San Francisco,2020-03-15
10,Station 10,37.781400,-122.421900,20,San Francisco,2020-10-05
17,Station 17,37.745300,-122.448600,20,San Francisco,2021-05-29
22,Station 22,37.717300,-122.465600,20,San Francisco,2021-10-27
6,Station 6,37.791000,-122.401600,19,San Francisco,2020-06-20
